In [1]:
import argparse
import time
import torch
print(torch.__version__)
from Models import get_model
from Process import *
import torch.nn.functional as F
from Optim import CosineWithRestarts
from Batch import create_masks
import dill as pickle
import time
import math
import numpy as np
import multiprocessing as mp
import random
import string
import sys
import os
import whoosh, glob, time, pickle
import whoosh.fields as wf
from whoosh.qparser import QueryParser
from whoosh import index
import threading
from whoosh import filedb
from whoosh.filedb.filestore import FileStorage
import numpy as np
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import multiprocessing as mp
import time
from InMemorySearch import *
from client import *
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from logger import Logger
logger = Logger('./logs')

1.0.0


/home/smsarwar/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


### Setting up the parameters. If you are running the notebook for the first time set load_vocab parameter to None. After the model starts running it will put the vocabulary in the vocab folder. If you want to use the model in the paper set relevance_training variable to 1, otherwise the model would be a simple transformer. 

In [3]:
parser = argparse.ArgumentParser()
#If we are working on small dataset
small = 0
#If we want to activate relevance based training
relevance_training = 1
dst = 'vocab'
if small==1:
    parser.add_argument('-src_data', type=str, default='data/italian_small.txt')
    parser.add_argument('-trg_data', type=str, default='data/english_small.txt')
    parser.add_argument('-trg_data_retrieval', type=str, default='data/english_retrieval.txt')

else:
    parser.add_argument('-src_data', type=str, default='data/italian.txt')
    parser.add_argument('-trg_data', type=str, default='data/english.txt')  
    parser.add_argument('-trg_data_retrieval', type=str, default='data/LATIMESTEXT2.txt')
    parser.add_argument('-rm_data', type=str, default='data/english_rm_dir.txt') ##

parser.add_argument('-src_lang', type=str, default='it')
parser.add_argument('-trg_lang', type=str, default='en')
parser.add_argument('-no_cuda', action='store_true')
parser.add_argument('-SGDR', action='store_true')
parser.add_argument('-epochs', type=int, default=10)
parser.add_argument('-d_model', type=int, default=200)
parser.add_argument('-n_layers', type=int, default=6)
parser.add_argument('-heads', type=int, default=8)
parser.add_argument('-dropout', type=int, default=0.1)
parser.add_argument('-batchsize', type=int, default=1000)
parser.add_argument('-printevery', type=int, default=5)
#if you are running for the first time set load_vocab to None. A vocabulary would be created in the vocab directory. 
parser.add_argument('-load_vocab', type=str, default='clir_it_en')

if relevance_training == 1:
    #if there exists a model_weights file 
    my_file = Path("weights/model_weights")
    if my_file.is_file():
        parser.add_argument('-load_weights', type=str, default='weights')
    else:
        parser.add_argument('-load_weights', type=str, default=None)
    parser.add_argument('-lr', type=int, default=0.01)
else: 
    my_file = Path("weights/model_weights")
    if my_file.is_file():
        parser.add_argument('-load_weights', type=str, default='weights')
    else:
        parser.add_argument('-load_weights', type=str, default=None)         
    parser.add_argument('-lr', type=int, default=0.0001)
    
parser.add_argument('-max_strlen', type=int, default=80)
parser.add_argument('-checkpoint', type=int, default=5)

opt = parser.parse_args(args=[])


In [4]:
def tokenizer(text):  # create a tokenizer function
    return text.split()
    
def create_fields(opt):    
    print("loading tokenizers...") 
    TRG = data.Field(lower=True, tokenize=tokenizer, init_token='<sos>', eos_token='<eos>')
    TRG_REL = data.Field(lower=True, tokenize=tokenizer, init_token='<sos>', eos_token='<eos>')    
    SRC = data.Field(lower=True, tokenize=tokenizer)   
    SRC = pickle.load(open(f'{opt.load_vocab}/SRC.pkl', 'rb'))
    TRG = pickle.load(open(f'{opt.load_vocab}/TRG.pkl', 'rb'))
    TRG_REL = pickle.load(open(f'{opt.load_vocab}/TRG.pkl', 'rb'))
    return(SRC, TRG, TRG_REL)


def create_dataset(opt, SRC, TRG, TRG_REL):
    """
    This function is used to construct the training batches from the input. 
    opt: parameters 
    SRC: source data for machine translation. 
    TRG: target data for machine translation. 
    TRG_REL: retrieval data for machine translation. This data is constructed apriori using TRG sentences as 
    queries to the retrieval corpus and retrieving a passage from the most relevant document. we call it relevance 
    data 
    """
    
    print("creating dataset and iterator... ")
    translation_data = [line for line in opt.trg_data] ### 
    relevance_data = [line for line in open(opt.rm_data)] ### relevance data is loaded from rm_data file.     
    
    raw_data = {'src' : [line.strip() for line in opt.src_data], 'trg': translation_data, 'trg_rel': relevance_data} ###  
    #raw data retrieval is the whole retrieval corpus. we use it to estimate tf-idf of all the words in the 
    #retrieval corpus. 
    
    raw_data_retrieval = {'trg': [line for line in opt.trg_data_retrieval]}
    df = pd.DataFrame(raw_data, columns=["src", "trg", "trg_rel"])
    
    #using the retrieval data to compute idf of every token in the retrieval corpus 
    vectorizer = TfidfVectorizer(use_idf=True)
    vectorizer.fit_transform(raw_data['trg'] + raw_data_retrieval['trg'])
    tokens = vectorizer.get_feature_names()
    idf_values = vectorizer.idf_
    opt.idf_dict = {}
    for i in range(len(tokens)):
        opt.idf_dict.setdefault(tokens[i],idf_values[i]) 
    
    #chopping off sentences with maximum string length parameter
    mask = (df['src'].str.count(' ') < opt.max_strlen) & (df['trg'].str.count(' ') < opt.max_strlen)
    df = df.loc[mask]
    
    #a dataframe is required for torchtext. we are creating a temporary one. each row of the dataframe would 
    #consist of source sentence, target sentence, and relevant passage retrieved using the target sentence. 
    df.to_csv("translate_transformer_temp.csv", index=False)
    
    data_fields = [('src', SRC), ('trg', TRG) , ('trg_rel', TRG_REL)]
    
    train = data.TabularDataset('./translate_transformer_temp.csv', format='csv', fields=data_fields)
    #creating training batches. 
    train_iter = MyIterator(train, batch_size=opt.batchsize, device=opt.device,
                        repeat=False, sort_key=lambda x: (len(x.src), len(x.trg)),
                        batch_size_fn=batch_size_fn, train=True, shuffle=True)
    #the following file is not necessary anymore.   
    os.remove('translate_transformer_temp.csv')   
    
    
    if opt.load_vocab is None:
        #if we have not built the vocabulary, we will have to do it here. We are integrating both the retrieval 
        #copus as well as the translation corpus. 
        print("creating dataset for retrieval corpus... ")
        raw_data = {'trg': [line for line in opt.trg_data_retrieval]}
        df = pd.DataFrame(raw_data, columns=["trg"])
        mask = (df['trg'].str.count(' ') > 1)
        df = df.loc[mask]
        df.to_csv("translate_transformer_retrieval_temp.csv", index=False)
        data_fields = [('trg', TRG)]
        train_retrieval = data.TabularDataset('./translate_transformer_retrieval_temp.csv', format='csv', fields=data_fields)
        os.remove('translate_transformer_retrieval_temp.csv')    

        print("building vocabulary for both europarl and retrieval corpus")

        SRC.build_vocab(train)
        TRG.build_vocab(train, train_retrieval)

    opt.src_pad = SRC.vocab.stoi['<pad>']
    opt.trg_pad = TRG.vocab.stoi['<pad>']
    
    opt.train_len = get_len(train_iter)
    return train_iter



In [5]:
opt.device = 0 if opt.no_cuda is False else -1
if opt.device == 0:
    assert torch.cuda.is_available()
print(opt.device)
read_data(opt)

SRC, TRG, TRG_REL = create_fields(opt)
opt.train = create_dataset(opt, SRC, TRG, TRG_REL)

#TRG = create_retrieval_vocabulary(opt, TRG)

if opt.checkpoint > 0:
    print("model weights will be saved every %d minutes and at end of epoch to directory weights/"%(opt.checkpoint))
    
if opt.load_vocab is None:
    pickle.dump(SRC, open(f'{dst}/SRC.pkl', 'wb'))
    pickle.dump(TRG, open(f'{dst}/TRG.pkl', 'wb'))

0
loading tokenizers...
creating dataset and iterator... 


The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


model weights will be saved every 5 minutes and at end of epoch to directory weights/


In [6]:
CONTEXT_SIZE = 2            
model = get_model(opt, len(SRC.vocab), len(TRG.vocab), CONTEXT_SIZE)
opt.optimizer = torch.optim.Adam(model.parameters(), lr=opt.lr, betas=(0.9, 0.98), eps=1e-9)
opt.optimizer_wemb = torch.optim.Adam(model.parameters(), lr = 0.000001, betas=(0.9, 0.98), eps=1e-9)
if opt.SGDR == True:
    opt.sched = CosineWithRestarts(opt.optimizer, T_max=opt.train_len)
    
print(TRG.vocab.itos[0])
print(len(TRG.vocab))
opt.idf_dict["<sos>"] = 1
opt.idf_dict["<eos>"] = 1
class_weights = []
for i in range(len(TRG.vocab)):
    if TRG.vocab.itos[i] in opt.idf_dict:
        class_weights.append(opt.idf_dict[TRG.vocab.itos[i]])
    else:
        class_weights.append(0.0001)
class_weights = torch.FloatTensor(class_weights)

loading pretrained weights...
<unk>
240229


In [7]:
print(TRG.vocab.itos[3])

<eos>


In [ ]:
from torch import nn 
import subprocess
opt.checkpoint = 5
nll_loss = nn.NLLLoss() # loss function
            
def train_model(model, opt, SRC, TRG):
    torch.cuda.empty_cache()
    best_mAP = 0.10
    print("training model...")
    opt.idf_dict["<sos>"] = 1
    opt.idf_dict["<eos>"] = 1
    
    #class weight indicates the weight of each term in calculating the loss function 
    #we have set it as the idf of the terms. 
    class_weights = []
    for i in range(len(TRG.vocab)):
        if TRG.vocab.itos[i] in opt.idf_dict:
            class_weights.append(opt.idf_dict[TRG.vocab.itos[i]])            
        else:
            class_weights.append(0.0001)
    
    class_weights = torch.FloatTensor(class_weights)
    model.train()
    class_weights = class_weights.cuda()
    
    start = time.time()
    #If we are checkpointing model after a certain period of time, we will use cptime to keep track of time
    if opt.checkpoint > 0:
        cptime = time.time()
        checkpointing_step = 0
        
    #the variable to keep track of global step 
    #print("number of batches: ".format(len(opt.train)))
    
    for epoch in range(opt.epochs):
        print("beginning epoch: {}".format(epoch))
        step = 0
        translation_loss = 0
        embedding_loss = 0
                
        for i, batch in enumerate(opt.train):
            #in each batch we have three inputs src, trg and trg_retrieval. This is the part where it is different 
            #from traditional machine translation. trg_retrieval comes from the retrieval corpus. please refer to the 
            #paper that for each sentence in trg we retrieve from a corpus and add it to the parallel data. 
            torch.cuda.empty_cache()
            start = time.clock()
            src = batch.src.transpose(0,1)
            trg = batch.trg.transpose(0,1)    
            trg_rel = batch.trg_rel.transpose(0,1) 
            #the length of trg is pretty small and that's why we are adding it three times and finally adding it with 
            #trg_rel. trg_rel comes from the retrieval corpus. This is why and how we construct tt. 
            tt = torch.cat((trg[: , :-1], trg[: , :-1], trg[: , :-1], trg_rel), 1).numpy()
            
            trg_input = trg[:, :-1]
            src_mask, trg_mask = create_masks(src, trg_input, opt)            
            src_mask = src_mask.cuda()            
            trg_mask = trg_mask.cuda()
            src = src.cuda() 
            trg_input = trg_input.cuda()  
            data_context_final = []
            data_target_final = []
            #now creating the training batches for word embedding on the fly. 
            for element in tt: 
                #removing the index of some unnecessay tokens
                indices = [i for i, x in enumerate(element) if x == 0 or x == 1 or x == 2 or x ==3]
                element = np.delete(element, indices)
                np.random.shuffle(element)
                corpus_text = torch.tensor(element)
                #creating word embedding data. The pivot will be at index i. 
                for i in range(CONTEXT_SIZE, len(corpus_text) - CONTEXT_SIZE):
                    #at first create the context 
                    data_context = []
                    data_target = []
                    for j in range(CONTEXT_SIZE):
                        data_context.append(corpus_text[i - CONTEXT_SIZE + j])
                    for j in range(1, CONTEXT_SIZE + 1):
                        data_context.append(corpus_text[i + j])
                    #now create the pivot or the target 
                    data_target.append(corpus_text[i])
                    #add context and pivot to create a batch
                    data_context_final.append(torch.LongTensor(data_context))
                    data_target_final.append(torch.LongTensor(data_target))
                    
            #Now we are training word embedding. 
            offset = 1000
            number_of_batches = int(len(data_context_final) / offset)
            index = 0
            loss_wemb = 0
            for batch in range(number_of_batches):
                opt.optimizer_wemb.zero_grad()
                #creating a batch of 1000 context, target pairs 
                data_context_final_temp = torch.stack(data_context_final[index:index+offset]).cuda()
                data_target_final_temp = torch.stack(data_target_final[index:index+offset]).cuda()            
                preds_emb = model(src, trg_input, src_mask, trg_mask, data_context_final_temp)
                #computing word embedding loss for 1000 data points for the word embedding task  
                loss_wemb_temp = F.cross_entropy(preds_emb.view(-1, preds_emb.size(-1)), data_target_final_temp.contiguous().view(-1).cuda())
                #make word embedding loss less impactful 
                loss_wemb_temp/= 10
                loss_wemb+=loss_wemb_temp.item()
                index+=1
                loss_wemb_temp.backward()
                opt.optimizer_wemb.step()
                
            #Now using the model to translate src to trg_input. 
            preds = model(src, trg_input, src_mask, trg_mask, 2)
            #The original translations 
            ys = trg[:, 1:].contiguous().view(-1).cuda()
            opt.optimizer.zero_grad()
            #computing cross-entropy loss for the translation task 
            loss = F.cross_entropy(preds.view(-1, preds.size(-1)), ys, weight=class_weights, ignore_index=opt.trg_pad)              
            loss.backward()

            translation_loss+= loss.item()
            embedding_loss+= loss_wemb/index

            opt.optimizer.step()
            
            if opt.SGDR == True: 
                opt.sched.step()
            
            print(str(step) + "\tstep loss nmt\t" + str(loss.item()) + "\tstep loss embedding\t" + str(loss_wemb/index))                
            step+=1
                
            #We are saving the model every five minutes. IR loss is costly to evaluate, hence we use this mechanism.
            if opt.checkpoint > 0 and ((time.time()-cptime)//60) // opt.checkpoint >= 1:
                #saving the model in a directory 
                torch.save(model.state_dict(), 'weights/model_weights')                            
                #loading the model to evaluate on IR task. 
                p = subprocess.Popen('CUDA_VISIBLE_DEVICES=3 python translate_validation.py', shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
                line = p.stdout.readlines()[0]
                #reading the evaluation from the IR task. specifically focusing on mAP. 
                mAP = float(line.decode("utf-8").strip().split()[1])
                print(str(checkpointing_step) + "\t"  + str(mAP) + "\tstep loss nmt\t" + str(translation_loss/step) + "\tstep loss embedding\t" + str(embedding_loss/step) + "\tcheckpoint info")                
                #if we have a better map compared to the current map, we save the model. 
                if mAP > best_mAP:
                    best_mAP = mAP
                    #The best model would be saved in weights directory if we get a better map on the validation set
                    torch.save(model.state_dict(), 'weights/model_weights_best_validation')
                cptime = time.time()
                info = { 'loss': loss.item(), 'map': mAP}
                for tag, value in info.items():
                    logger.scalar_summary(tag, value, checkpointing_step+1)
                checkpointing_step+=1
            
        print(str(opt.epoch) + "\tepoch loss nmt\t" + str(translation_loss/step) + "\tepoch loss embedding\t" + str(embedding_loss/step) + "\tepoch info")                
                
        #Finally saving a model after an epoch. 
        torch.save(model.state_dict(), 'weights/model_weights_' + str(epoch))

In [ ]:
#torch.cuda.empty_cache()
model = model.cuda()
train_model(model, opt, SRC, TRG)

training model...
beginning epoch: 0
0	step loss nmt	3.9221818447113037	step loss embedding	1.2294775992631912
1	step loss nmt	16.530879974365234	step loss embedding	1.2403833184923445
2	step loss nmt	31.954986572265625	step loss embedding	1.2416691184043884
3	step loss nmt	25.41067123413086	step loss embedding	1.2426793575286865
4	step loss nmt	18.872697830200195	step loss embedding	1.2458011209964752
5	step loss nmt	18.58994483947754	step loss embedding	1.2513601660728455
6	step loss nmt	21.455183029174805	step loss embedding	1.2514421790838242
7	step loss nmt	20.14837646484375	step loss embedding	1.245383381843567
8	step loss nmt	19.965816497802734	step loss embedding	1.2517280578613281
9	step loss nmt	18.539390563964844	step loss embedding	1.2480195210530207
10	step loss nmt	17.99416160583496	step loss embedding	1.2518243619373866
11	step loss nmt	18.297880172729492	step loss embedding	1.248568868637085
12	step loss nmt	17.861736297607422	step loss embedding	1.2507413983345033
13	s

108	step loss nmt	9.96865177154541	step loss embedding	1.3064419407593577
109	step loss nmt	9.93358039855957	step loss embedding	1.3000219662984211
110	step loss nmt	10.18037223815918	step loss embedding	1.3111427545547485
111	step loss nmt	10.347564697265625	step loss embedding	1.3358052521944046
112	step loss nmt	10.064628601074219	step loss embedding	1.2856353861945016
113	step loss nmt	9.831003189086914	step loss embedding	1.2958697199821472
114	step loss nmt	10.234359741210938	step loss embedding	1.2918894290924072
115	step loss nmt	9.895061492919922	step loss embedding	1.2836664219697316
116	step loss nmt	11.070008277893066	step loss embedding	1.3028392493724823
117	step loss nmt	9.843363761901855	step loss embedding	1.3119183381398518
118	step loss nmt	9.878853797912598	step loss embedding	1.3277562686375208
119	step loss nmt	9.655867576599121	step loss embedding	1.3057878255844115
120	step loss nmt	10.056974411010742	step loss embedding	1.2892587184906006
121	step loss nmt	9.59

216	step loss nmt	9.24958324432373	step loss embedding	1.2648679993369363
217	step loss nmt	9.236075401306152	step loss embedding	1.2664123406777015
218	step loss nmt	9.618104934692383	step loss embedding	1.3248342672983806
219	step loss nmt	8.247587203979492	step loss embedding	1.3430165290832519
220	step loss nmt	8.887136459350586	step loss embedding	1.2844339098249162
221	step loss nmt	9.481388092041016	step loss embedding	1.269804798066616
222	step loss nmt	9.146309852600098	step loss embedding	1.2805055873147373
223	step loss nmt	9.503437995910645	step loss embedding	1.2905791997909546
224	step loss nmt	8.901814460754395	step loss embedding	1.2569333579805162
225	step loss nmt	9.160184860229492	step loss embedding	1.2715411828114436
226	step loss nmt	9.30960464477539	step loss embedding	1.3162663221359252
227	step loss nmt	8.710799217224121	step loss embedding	1.3486074341668024
228	step loss nmt	9.60727596282959	step loss embedding	1.2651828697749548
229	step loss nmt	8.831098556

324	step loss nmt	8.772494316101074	step loss embedding	1.3091284831364949
325	step loss nmt	8.87333869934082	step loss embedding	1.3382326066493988
326	step loss nmt	8.963475227355957	step loss embedding	1.3123536556959152
327	step loss nmt	8.73640251159668	step loss embedding	1.3226934969425201
328	step loss nmt	8.621338844299316	step loss embedding	1.297798901796341
329	step loss nmt	8.557950973510742	step loss embedding	1.2968035650253296
330	step loss nmt	8.673569679260254	step loss embedding	1.3587224781513214
331	step loss nmt	8.526082038879395	step loss embedding	1.2585987011591593
332	step loss nmt	8.859979629516602	step loss embedding	1.3613766431808472
333	step loss nmt	8.682827949523926	step loss embedding	1.3200665354728698
334	step loss nmt	8.684432029724121	step loss embedding	1.2904251151614718
335	step loss nmt	8.741538047790527	step loss embedding	1.265244537386401
336	step loss nmt	8.671205520629883	step loss embedding	1.2889096112478347
337	step loss nmt	8.894404411

432	step loss nmt	8.770748138427734	step loss embedding	1.3126161098480225
433	step loss nmt	8.430312156677246	step loss embedding	1.3369815826416016
434	step loss nmt	8.261602401733398	step loss embedding	1.3307399253050487
435	step loss nmt	8.231709480285645	step loss embedding	1.4090728878974914
436	step loss nmt	8.438840866088867	step loss embedding	1.250011682510376
437	step loss nmt	8.345796585083008	step loss embedding	1.3035880785721998
438	step loss nmt	8.45858097076416	step loss embedding	1.3122703954577446
439	step loss nmt	8.265209197998047	step loss embedding	1.423964023590088
440	step loss nmt	8.21714973449707	step loss embedding	1.2701641504581158
441	step loss nmt	8.561320304870605	step loss embedding	1.2910556684840808
442	step loss nmt	8.241265296936035	step loss embedding	1.2703839865597812
443	step loss nmt	8.433807373046875	step loss embedding	1.324491646554735
444	step loss nmt	8.524564743041992	step loss embedding	1.3784843921661376
445	step loss nmt	8.2643108367

...

2139	step loss nmt	7.596896171569824	step loss embedding	2.197924256324768
2140	step loss nmt	7.286527156829834	step loss embedding	2.013569434483846
2141	step loss nmt	7.766252040863037	step loss embedding	2.088985482851664
2142	step loss nmt	8.197171211242676	step loss embedding	2.0138111764734443
2143	step loss nmt	7.490690231323242	step loss embedding	2.063269903785304
2144	step loss nmt	7.983999729156494	step loss embedding	2.255767128684304
2145	step loss nmt	8.203299522399902	step loss embedding	2.1230635378095837
2146	step loss nmt	8.938161849975586	step loss embedding	2.088732375038995
2147	step loss nmt	8.079096794128418	step loss embedding	2.1108581318574795
2148	step loss nmt	7.821846961975098	step loss embedding	2.1341744899749755
2149	step loss nmt	7.985294818878174	step loss embedding	2.232776509390937
2150	step loss nmt	7.5702948570251465	step loss embedding	2.0736790895462036
2151	step loss nmt	8.000778198242188	step loss embedding	2.194398456149631
2152	step loss nmt	

2246	step loss nmt	7.895325660705566	step loss embedding	2.1716105143229165
2247	step loss nmt	7.7174482345581055	step loss embedding	2.1816953145540676
2248	step loss nmt	8.110840797424316	step loss embedding	2.275731384754181
2249	step loss nmt	7.823238372802734	step loss embedding	2.1154610216617584
2250	step loss nmt	8.021939277648926	step loss embedding	2.083923101425171
2251	step loss nmt	7.833564758300781	step loss embedding	2.1811193943023683
2252	step loss nmt	8.507272720336914	step loss embedding	2.3047703829678623
2253	step loss nmt	8.016098022460938	step loss embedding	2.336564336504255
2254	step loss nmt	8.125518798828125	step loss embedding	2.1881739876487036
2255	step loss nmt	7.952013969421387	step loss embedding	2.065101639429728
2256	step loss nmt	8.4173583984375	step loss embedding	2.234973464693342
2257	step loss nmt	7.625447750091553	step loss embedding	2.185197591781616
2258	step loss nmt	7.747110366821289	step loss embedding	2.239203748703003
2259	step loss nmt	7

2352	step loss nmt	7.4588494300842285	step loss embedding	2.204971262386867
2353	step loss nmt	8.047195434570312	step loss embedding	2.19264946381251
2354	step loss nmt	7.496706962585449	step loss embedding	2.2004595597585044
2355	step loss nmt	5.800051212310791	step loss embedding	2.200084238052368
2356	step loss nmt	7.332099437713623	step loss embedding	2.17044338313016
2357	step loss nmt	8.180510520935059	step loss embedding	2.3041759729385376
2358	step loss nmt	7.879012584686279	step loss embedding	2.1348236448624553
2359	step loss nmt	7.746572017669678	step loss embedding	2.0508896020742564
2360	step loss nmt	8.202258110046387	step loss embedding	2.0793644189834595
2361	step loss nmt	7.7867584228515625	step loss embedding	2.170742188181196
2362	step loss nmt	7.579340934753418	step loss embedding	2.0058558305104572
2363	step loss nmt	7.764938831329346	step loss embedding	2.1259970029195148
2364	step loss nmt	7.571615695953369	step loss embedding	2.2089510122934977
2365	step loss nm

2458	step loss nmt	7.255510330200195	step loss embedding	2.316431482632955
2459	step loss nmt	7.6133036613464355	step loss embedding	2.4622477144002914
2460	step loss nmt	7.717666149139404	step loss embedding	2.178651054700216
2461	step loss nmt	7.9741339683532715	step loss embedding	2.196063533425331
2462	step loss nmt	8.122941017150879	step loss embedding	2.341081460316976
2463	step loss nmt	7.794125080108643	step loss embedding	2.2291197299957277
2464	step loss nmt	7.699281692504883	step loss embedding	2.31863109767437
2465	step loss nmt	8.013415336608887	step loss embedding	2.2258122126261393
2466	step loss nmt	7.613790512084961	step loss embedding	2.209941118955612
2467	step loss nmt	7.749589443206787	step loss embedding	2.4468262412331323
2468	step loss nmt	7.301813125610352	step loss embedding	2.342943640316234
2469	step loss nmt	7.729976177215576	step loss embedding	2.2120742052793503
2470	step loss nmt	8.10204029083252	step loss embedding	2.207317900657654
2471	step loss nmt	7

2564	step loss nmt	8.714007377624512	step loss embedding	2.1982717911402383
2565	step loss nmt	8.331901550292969	step loss embedding	2.2765616178512573
2566	step loss nmt	7.625314712524414	step loss embedding	2.0808005954908286
2567	step loss nmt	7.803764820098877	step loss embedding	2.1872385342915854
2568	step loss nmt	7.675093173980713	step loss embedding	2.2354053258895874
2569	step loss nmt	7.386745452880859	step loss embedding	2.2100587541406806
2570	step loss nmt	7.680424690246582	step loss embedding	2.269737958908081
2571	step loss nmt	7.521056175231934	step loss embedding	2.2602350314458213
2572	step loss nmt	7.922428607940674	step loss embedding	2.348035156726837
2573	step loss nmt	8.061807632446289	step loss embedding	2.332236797913261
2574	step loss nmt	7.83349084854126	step loss embedding	2.1914816796779633
2575	step loss nmt	7.858027458190918	step loss embedding	2.240982549531119
2576	step loss nmt	7.8176727294921875	step loss embedding	2.2862598385129655
2577	step loss n

In [ ]:
torch.save(model.state_dict(), f'{load_vocab}/model_weights')

In [ ]:
print(SRC.vocab.itos[4880])